In [10]:
import sys
sys.path.append("./work")

In [11]:
raw_data_path = "./work/data/knmi.parquet"
output_directory_path = "./work/output/"
considered_cities = ["De Bilt"]
start_date = "2003-01-01"

In [12]:
from pyspark.sql import SparkSession

In [13]:
spark = SparkSession.builder.appName("ColdwaveAnalysis").getOrCreate()

In [14]:
df = spark.read.parquet(raw_data_path, header=True, inferSchema=True)

In [15]:
import src.common as c
import src.coldwaves as cold


df = c.filter_needed_data(df, considered_cities, start_date)
df = c.clean_data(df)
df = c.with_date(df)
df = c.aggregate_daily(df)
df = cold.with_minimum_flags(df)
df = cold.filter_wave_candidates(df)
df = c.with_prev_date(df)
df = c.with_date_diff(df)
df = c.with_group(df)
df = cold.group_metrics(df)
df = cold.filter_heatwaves(df)

In [16]:
from pyspark.sql import functions as F


(df
 .select(
    F.col("FROM_DATE"),
    F.col("TO_DATE"),
    F.col("DURATION"),
    F.col("HIGH_FROST_DAYS"),
    F.col("MIN_TEMPERATURE"),
)
    .orderBy(F.col("FROM_DATE"))
    .coalesce(1)
    .write
    .option("header", "true")
    .csv(output_directory_path + "coldwaves.csv")
)